# Redes Convolucionadas

## Ejemplo mnist - handwrite numbers

## Autor

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 


## Referencias



# El modelo

## Importa módulos

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Versión de Tensorflow: ", tf.__version__)

## Lee los datos desde tf.keras y los preprocesa

In [ ]:
# apunta  los datos
mnist = tf.keras.datasets.mnist

# lee los conjuntos de datos, los cuales vienen separados de antemano para entrenamiento y test
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# reshape para colocar los datos de entrenamiento en el fomato apropiado. Agrega una dimensión al final y  normaliza los datos
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
# reshape para colocar los datos de test en el fomato apropiado. Agrega una dimensión al final y  normaliza los datos
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

In [ ]:
training_images.shape

In [ ]:
# muestra una imagen de entranamiento
import matplotlib.pyplot as plt
print(training_images[10,:,:,0])



In [ ]:
print('label =' ,training_labels[10])
plt.imshow(training_images[10,:,:,0],cmap='gray')
plt.show()

## Define el modelo

1. Capa 1: Conv2D con 32 filtros (kernels) de tamaño 3*3. Cada image viene en un tensor de tamaño 28*28*1. Los filtros son pasados por una f. de activaci+on 'relu'.
2. Capa 2. MaxPooling. Reduce cada filtro. Toma regiones secuenciales 2*2 y los reduce tomando el máximo de cada región. No usa f. de activación nunca.

<figure>
<center>
<img src="./Imagenes/maxpool.webp" width="600" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Max pooling 2*2</p>
</figcaption>
</figure>

3. Capa 3. Flatten. Toma todos los filtros resultantes  de la capa MaxPooling y los organiza como un único tensor unidimensional
4. Capa 4. Dense. Recibe el tensor saliente de la capa Flatten y genera una salida en 128 unidades, usando activación *relu*
5. Capa 5. Densa Recibe el tensor unidmensional de shape=128 t lo transforma en la salida de 10 unidades. Hay 10 clases. El predictor es al final transformado por la acticación *softmax* para obtener una distribución de la posible clase para la imagend e entrada

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

# Entrenamiento

## Crea una clase derivada de tf.keras.callbacks.Callback 

Se usa para pasar funciones de control al algoritmo de estimación. Aquí la usaremos para el entrenamiento pare cuando se alcance un determinado accuracy con los datos de entrenamiento

*tf.keras.callbacks.Callback* es una clase abstracta para permitir escribir métodos que actuan en el proceso de entranamiento o de test. Para detalles vea [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback).

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.999):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

# crea una instancia de clase
accu_callback = myCallback()


## Crea un checkpoint para guardar los pesos

In [ ]:
# Directory where the checkpoints will be saved
import os 
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "mnist_ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True, 
    monitor='val_accuracy', mode='max',
    save_best_only=True)

## Compila el modelo

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Observe como se pasan los callbacks

## Entrena el modelo

In [ ]:

#history = model.fit(training_images, training_labels, epochs=100, validation_split=0.2, batch_size=32,callbacks=[accu_callback, checkpoint_callback])
history = model.fit(training_images, training_labels, epochs=20, validation_split=0.2, batch_size=64,callbacks=[accu_callback,checkpoint_callback ])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 29s 609us/sample - loss: 1.0820e-06 - accuracy: 1.0000 - val_loss: 0.0970 - val_accuracy: 0.9880
Epoch 2/20
48000/48000 [==============================] - 30s 626us/sample - loss: 8.0284e-07 - accuracy: 1.0000 - val_loss: 0.0993 - val_accuracy: 0.9887
Epoch 3/20
48000/48000 [==============================] - 31s 641us/sample - loss: 5.6646e-07 - accuracy: 1.0000 - val_loss: 0.1020 - val_accuracy: 0.9883
Epoch 4/20
22080/48000 [============>.................] - ETA: 14s - loss: 3.9061e-07 - accuracy: 1.0000

# Evalua el modelo

# Gráficas de funciones de pérdida y accuracy

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

n_row = 1
n_col = 2
fig, ax = plt.subplots(n_row, n_col, sharex = False, sharey = False, figsize=(16,4))


ax[0].plot(epochs, acc, 'r', label='Training accuracy')
ax[0].plot(epochs, val_acc, 'b', label='Validation accuracy')
ax[0].legend(fontsize=12,loc=0)
ax[0].set_title('Training and Validation Accuracy',fontsize=16)
ax[0].set_ylabel('measure',fontsize=14)
ax[0].set_xlabel('epoch', fontsize = 14)
ax[0].set_xlim([1, len(acc)])

ax[1].plot(epochs, loss, 'r', label='Training Loss')
ax[1].plot(epochs, val_loss, 'b', label='Validation Loss')
ax[1].legend(fontsize=12)
ax[1].set_title('Training and Validation Loss',fontsize=16)
ax[1].set_ylabel('measure',fontsize=14)
ax[1].set_xlabel('epoch', fontsize = 14)
ax[1].set_xlim([1, len(acc)])


plt.show()

## Carga los pesos del mejor modelo obtenido

In [ ]:
#model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

#model.build(tf.TensorShape([1, None]))

## Evalua el modelo con los dato de prueba (test)

In [ ]:
model.evaluate(test_images,test_labels)

## Muestra de algunas predicciones

Calcula las clases predichas las probabilidades calculadas por softmax

In [86]:
preds = model.predict_classes(test_images)
prob = model.predict_proba(test_images)
len(preds)

10000


In [94]:

for i in range(100):
    print (i, preds[i], test_labels[i])


0 7 7
1 2 2
2 1 1
3 0 0
4 4 4
5 1 1
6 4 4
7 9 9
8 5 5
9 9 9
10 0 0
11 6 6
12 9 9
13 0 0
14 1 1
15 5 5
16 9 9
17 7 7
18 5 3
19 4 4
20 9 9
21 6 6
22 6 6
23 5 5
24 4 4
25 0 0
26 7 7
27 4 4
28 0 0
29 1 1
30 3 3
31 1 1
32 3 3
33 4 4
34 7 7
35 2 2
36 7 7
37 1 1
38 2 2
39 1 1
40 1 1
41 7 7
42 4 4
43 2 2
44 3 3
45 5 5
46 1 1
47 2 2
48 4 4
49 4 4
50 6 6
51 3 3
52 5 5
53 5 5
54 6 6
55 0 0
56 4 4
57 1 1
58 9 9
59 5 5
60 7 7
61 8 8
62 9 9
63 3 3
64 7 7
65 4 4
66 6 6
67 4 4
68 3 3
69 0 0
70 7 7
71 0 0
72 2 2
73 9 9
74 1 1
75 7 7
76 3 3
77 2 2
78 9 9
79 7 7
80 7 7
81 6 6
82 2 2
83 7 7
84 8 8
85 4 4
86 7 7
87 3 3
88 6 6
89 1 1
90 3 3
91 6 6
92 9 9
93 3 3
94 1 1
95 4 4
96 1 1
97 7 7
98 6 6
99 9 9


In [ ]:
## imagen 18

In [ ]:
print('label =' ,training_labels[10])
plt.imshow(training_images[10,:,:,0],cmap='gray')
plt.show()

## detección de malas predicciones

In [96]:
bad_pred=[]
for i in range (len(preds)):
    if preds[i]!=test_labels[i]:
        bad_pred.append(i)

In [97]:
bad_pred

[18,
 321,
 340,
 445,
 659,
 740,
 882,
 883,
 938,
 965,
 1014,
 1039,
 1156,
 1182,
 1226,
 1232,
 1242,
 1247,
 1319,
 1378,
 1393,
 1459,
 1530,
 1621,
 1678,
 1686,
 1709,
 1717,
 1790,
 1901,
 1955,
 2035,
 2070,
 2118,
 2130,
 2135,
 2293,
 2387,
 2414,
 2437,
 2488,
 2597,
 2654,
 2896,
 2921,
 2927,
 2953,
 3060,
 3073,
 3206,
 3225,
 3422,
 3451,
 3503,
 3520,
 3558,
 3597,
 3727,
 3767,
 3778,
 3796,
 3808,
 3906,
 4140,
 4176,
 4201,
 4248,
 4443,
 4578,
 4740,
 4807,
 4814,
 4823,
 4860,
 5331,
 5634,
 5887,
 5937,
 5955,
 6555,
 6571,
 6576,
 6597,
 8094,
 8325,
 8408,
 9009,
 9015,
 9019,
 9587,
 9634,
 9642,
 9664,
 9679,
 9698,
 9729,
 9768,
 9792,
 9839,
 9982]

In [ ]:
print('label =' ,training_labels[bad_pred[2]], 'prediction= ',preds[bad_pred[2]])
plt.imshow(training_images[bad_pred[2],:,:,0],cmap='gray')
plt.show()